In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.regularizers import l1,l2
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
import os
import random as rn
import numpy as np
import logging
import time
from util_functions import *

# disable GPUs for test reproducibility
tf.config.set_visible_devices([], 'GPU')

SEED=0

# DATASET_FOLDER = "./DOS2019_Binary"

# # Load your dataset (for example, Iris dataset)
# data = load_iris()
# X, y = data.data, data.target

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')
# X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5')
# X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5')

DATASET_FOLDER = "./Dataset"

X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5',channels=True)
X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5',channels=True)
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5',channels=True)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

(3669, 10, 11, 1) (3669,)
(453, 10, 11, 1) (453,)
(408, 10, 11, 1) (408,)


2024-12-20 08:48:05.367304: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-12-20 08:48:05.367348: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: jarvis
2024-12-20 08:48:05.367355: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: jarvis
2024-12-20 08:48:05.367441: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.120.0
2024-12-20 08:48:05.367461: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.120.0
2024-12-20 08:48:05.367466: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.120.0


In [2]:
def compileModel(model,optimizer='sgd', lr=0.001):
    if optimizer == 'sgd':
        optimizer = SGD(learning_rate=lr, momentum=0.0)
    else:
        optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy']) 

In [3]:
# Function to create the MLP model (for GridSearchCV)
def create_model(optimizer='sgd', dense_layers=4, hidden_units=2, learning_rate = 0.001, dropout_rate=0, activation='relu'):
    model = Sequential(name  = "mlp")
    model.add(Dense(hidden_units, input_shape=(21,), activation='relu'))
    model.add(Dropout(dropout_rate))
    for layer in range(dense_layers):
        model.add(Dense(hidden_units, activation='relu', name='hidden-fc' + str(layer)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid', name='fc2'))
    compileModel(model, optimizer,learning_rate)
    return model

In [4]:
# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)

PATIENCE = 10
k=2 # number of folds for cross-validation

# Define the hyperparameters to tune and their possible values
param_grid = {
    'learning_rate' : [0.001,0.01],
    'dense_layers' : [2,3],
    'hidden_units' : [32,64],
    'dropout_rate' : [0.2,0.5],
    'activation' : ['relu','tanh'],
    'batch_size' : [100,200],
    'optimizer' : ['sgd','adam']
}

# Perform grid search with 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=k)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=PATIENCE, verbose=1, mode='max', restore_best_weights=True)
start_time = time.time()

### ADD early stopping in the list of callbacks
grid_result = grid.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks= [])
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)
# Print the best parameters and corresponding accuracy
print("Best parameters found: ", grid_result.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(grid_result.best_score_))

# Evaluate the best model on the test set
best_model = grid.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test accuracy of the best model: {:.2f}".format(test_accuracy))

/tmp/ipykernel_38464/801539327.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)
2024-12-20 08:48:16.521891: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoc

ValueError: 
All the 256 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
256 fits failed with the following error:
Traceback (most recent call last):
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 232, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 164, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/keras/utils/traceback_utils